# Shopping Cart Demo

This demo shows how to use Webtask to automate adding items to a shopping cart with all three core methods:
- `do()` - Execute tasks
- `extract()` - Get information from pages
- `verify()` - Check conditions

## Setup

In [1]:
from webtask import Webtask
from webtask.integrations.llm import Gemini
import os

# Initialize
wt = Webtask()

# Create stateful agent (remembers context across tasks)
llm = Gemini(model="gemini-2.5-flash", api_key=os.getenv("GOOGLE_API_KEY"))
agent = await wt.create_agent(llm=llm)

## Task 1: Add Flat-Head Wood Screws

In [2]:
await agent.goto("https://practicesoftwaretesting.com/")

In [3]:
result1 = await agent.do("add 2 Flat-Head Wood Screws to the cart", wait_after_action=2)

print(f"Feedback: {result1.feedback}")

2025-12-05 15:53:23 - webtask._internal.agent.task_runner - INFO - Task start - Task: add 2 Flat-Head Wood Screws to the cart
2025-12-05 15:53:23 - webtask._internal.agent.task_runner - INFO - Step 1 - Start
2025-12-05 15:53:23 - webtask._internal.agent.task_runner - DEBUG - Sending LLM request...
2025-12-05 15:53:25 - webtask.llm.llm - INFO - Token usage - Prompt: 4409, Response: 45, Total: 4538
2025-12-05 15:53:25 - webtask._internal.agent.task_runner - INFO - Received LLM response - Tools: ['type', 'click']
2025-12-05 15:53:25 - webtask._internal.agent.tool_registry - INFO - Executing tool: type with params: {'clear': True, 'text': 'Flat-Head Wood Screws'}
2025-12-05 15:53:29 - webtask._internal.agent.tool_registry - INFO - Tool executed successfully: Typed 'Flat-Head Wood Screws' (cleared first)
2025-12-05 15:53:29 - webtask._internal.agent.tool_registry - INFO - Executing tool: click with params: {'element_id': 'button-4', 'description': 'Search button'}
2025-12-05 15:53:31 - webt

## Task 2: Add Cross-Head Screws

In [ ]:
result2 = await agent.do("Add 5 Cross-head screws to the cart")

print(f"Feedback: {result2.feedback}")

## Task 3: Extract Cart Information

Use the `extract()` method to get information from the page.

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class CartItem(BaseModel):
    """A single item in the shopping cart."""
    name: str = Field(description="Product name")
    quantity: int = Field(description="Quantity of this item")

class CartSummary(BaseModel):
    """Shopping cart contents."""
    items: List[CartItem] = Field(description="List of items in the cart")
    total_items: int = Field(description="Total number of items in cart")

# Navigate to cart and extract structured data
await agent.do("Go to the cart page")

# Simple extraction (returns string)
total = await agent.extract("total price")
print(f"Total Price: {total}")

# Structured extraction (returns Pydantic model)
cart = await agent.extract("cart items and total count", CartSummary)

print(f"\nTotal Items: {cart.total_items}")
print("\nCart Items:")
for item in cart.items:
    print(f"  - {item.name}: {item.quantity}")

## Task 4: Verify Cart Contents

Use the `verify()` method for simple boolean checks.

In [ ]:
verdict = await agent.verify("the cart contains exactly 7 items")

print(f"Passed: {verdict.passed}")
print(f"Feedback: {verdict.feedback}")

# Can use in boolean context
if verdict:
    print("\nVerification passed!")
else:
    print("\nVerification failed!")

# Can compare with boolean
assert verdict == True

## Error Handling

All three methods throw custom exceptions on abort.

In [ ]:
from webtask import TaskAbortedError, VerificationAbortedError, ExtractionAbortedError

# Example of error handling
try:
    await agent.do("Do something impossible")
except TaskAbortedError as e:
    print(f"Task failed: {e}")

## Cleanup

In [ ]:
await wt.close()